In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import datetime as dt

import matplotlib.pyplot as plt
from matplotlib import colors
import soundfile as sf
import matplotlib.patches as patches
from pathlib import Path

import dask.dataframe as dd

In [2]:
import sys

sys.path.append("../src")
print(sys.path)

['/Users/adityakrishna/duty-cycle-investigation/daily_notebook', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python311.zip', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python3.11', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python3.11/lib-dynload', '', '/Users/adityakrishna/miniconda3/envs/dc-study/lib/python3.11/site-packages', '../src']


In [3]:
from core import SITE_NAMES, DC_COLOR_MAPPINGS, FREQ_GROUPS
from cli import get_file_paths

import clustering as bt_clustering
import bout as bt
import plot as bt_plt
import activity.subsampling as ss
import activity.activity_assembly as dh
import activity.plot as plotting

In [4]:
import pipeline

In [5]:
type_key = ''
site_key = "Carp"
# cycle_lengths = [1800, 720, 600, 360, 180, 60, 30, 6]
cycle_lengths = [1800, 360]
percent_ons = [0.1667]
specific_dc_tag = "1800of1800"

In [6]:
fig_details = dict()
fig_details['site_name'] = SITE_NAMES[site_key]

In [7]:
pipeline_params = dict()
pipeline_params['assemble_location_summary'] = True
pipeline_params["read_csv"] = False
pipeline_params["save_activity_grid"] = True
pipeline_params["save_presence_grid"] = True
pipeline_params["save_dc_night_comparisons"] = True
pipeline_params["save_activity_dc_comparisons"] = True
pipeline_params["save_presence_dc_comparisons"] = True
pipeline_params["show_plots"] = False
pipeline_params["show_PST"] = True
pipeline_params

{'assemble_location_summary': True,
 'read_csv': False,
 'save_activity_grid': True,
 'save_presence_grid': True,
 'save_dc_night_comparisons': True,
 'save_activity_dc_comparisons': True,
 'save_presence_dc_comparisons': True,
 'show_plots': False,
 'show_PST': True}

In [8]:
data_params = dict()
data_params["site_name"] = SITE_NAMES[site_key]
data_params["site_tag"] = site_key
data_params["type_tag"] = type_key
data_params["cycle_lengths"] = cycle_lengths
data_params["percent_ons"] = percent_ons
dc_tags = ss.get_list_of_dc_tags(data_params["cycle_lengths"], data_params["percent_ons"])
data_params["dc_tags"] = dc_tags
data_params["cur_dc_tag"] = specific_dc_tag
data_params['resolution_in_min'] = '30'
data_params['recording_start'] = '01:00'
data_params['recording_end'] = '15:00'
data_params

{'site_name': 'Carp Pond',
 'site_tag': 'Carp',
 'type_tag': '',
 'cycle_lengths': [1800, 360],
 'percent_ons': [0.1667],
 'dc_tags': ['1800of1800', '300of1800', '60of360'],
 'cur_dc_tag': '1800of1800',
 'resolution_in_min': '30',
 'recording_start': '01:00',
 'recording_end': '15:00'}

In [9]:
file_paths = get_file_paths(data_params)
file_paths

{'raw_SITE_folder': '/Users/adityakrishna/duty-cycle-investigation/src/../data/raw/Carp',
 'SITE_folder': '/Users/adityakrishna/duty-cycle-investigation/src/../data/2022_bd2_summary/Carp',
 'bd2_TYPE_SITE_YEAR': 'bd2__Carp_2022',
 'duty_cycled_folder': '/Users/adityakrishna/duty-cycle-investigation/src/../data/2022_bd2_summary/Carp/duty_cycled',
 'dc_dets_TYPE_SITE_summary': 'dc_dets_Carp_summary',
 'dc_bouts_TYPE_SITE_summary': 'dc_bouts_Carp_summary',
 'dc_inds_TYPE_SITE_summary': 'dc_inds_Carp_summary',
 'simulated_schemes_folder': '/Users/adityakrishna/duty-cycle-investigation/src/../data/2022_bd2_summary/Carp/duty_cycled/simulated_schemes',
 'figures_SITE_folder': '/Users/adityakrishna/duty-cycle-investigation/src/../figures/Carp',
 'activity_det_comparisons_figname': 'activity_det_comparisons_per_dc_Carp',
 'dc_det_comparisons_figname': 'dc_det_comparisons_per_night_Carp',
 'activity_bout_comparisons_figname': 'activity_bout_comparisons_per_dc_Carp',
 'dc_bout_comparisons_figname

In [28]:
dc_tag = data_params['cur_dc_tag']

lf1_means = []
hf1_means = []
hf2_means = []
# for site_key in SITE_NAMES.keys():
site_key = 'Carp'
print(site_key)
data_params["site_name"] = SITE_NAMES[site_key]
data_params["site_tag"] = site_key
file_paths = get_file_paths(data_params)
if (Path(f'{file_paths["SITE_folder"]}/{file_paths["bd2_TYPE_SITE_YEAR"]}.csv').is_file()):
    location_df = ss.prepare_summary_for_plotting_with_duty_cycle(file_paths, dc_tag)
    bout_metrics = bt.generate_bout_metrics_for_location_and_freq(location_df, data_params, dc_tag)
    lf1_means.append(bout_metrics[bout_metrics['freq_group']=='LF1'].bout_duration.mean())
    hf1_means.append(bout_metrics[bout_metrics['freq_group']=='HF1'].bout_duration.mean())
    hf2_means.append(bout_metrics[bout_metrics['freq_group']=='HF2'].bout_duration.mean())

Carp


In [26]:
location_means = dict()
for i, site_key in enumerate(list(SITE_NAMES.keys())[:4]):
    print(i)
    group_means = dict()
    group_means['LF1'] = lf1_means[i]
    group_means['HF1'] = hf1_means[i]
    group_means['HF2'] = hf2_means[i]
    location_means[site_key] = group_means

0
1
2
3


In [27]:
location_means

{'Central': {'LF1': Timedelta('0 days 00:00:07.796126540'),
  'HF1': Timedelta('0 days 00:00:02.860802097'),
  'HF2': Timedelta('0 days 00:00:01.985767506')},
 'Foliage': {'LF1': Timedelta('0 days 00:00:04.067990277'),
  'HF1': Timedelta('0 days 00:00:02.830471955'),
  'HF2': Timedelta('0 days 00:00:02.050604854')},
 'Carp': {'LF1': Timedelta('0 days 00:00:10.683502006'),
  'HF1': Timedelta('0 days 00:00:02.946942213'),
  'HF2': Timedelta('0 days 00:00:01.856083963')},
 'Telephone': {'LF1': Timedelta('0 days 00:00:01.548397754'),
  'HF1': Timedelta('0 days 00:00:04.687656705'),
  'HF2': Timedelta('0 days 00:00:04.570759147')}}

In [16]:
np.array(lf1_means).mean()

Timedelta('0 days 00:00:06.024004144')

In [17]:
np.array(hf1_means).mean()

Timedelta('0 days 00:00:03.331468242')

In [19]:
np.array(hf2_means).mean()

Timedelta('0 days 00:00:02.615803867')